# Imports

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
from sampler_utils import retrieve_autoencoder_components_state_dicts, latent_ids_to_onehot3d, get_latent_loaders
from models3d import VQAutoEncoder, Generator
import matplotlib.pyplot as plt
from ipywidgets import interact, fixed
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from concurrent.futures import ThreadPoolExecutor
from PIL import Image
import torch.distributions as dists
from tqdm import tqdm
import gc
from models3d import BiomeClassifier



In [2]:
%matplotlib widget

# Pyvista Visualizer


In [3]:
# import pyvista as pv
# import matplotlib.cm as cm  # Add this import


# class MinecraftVisualizerPyVista:
#     def __init__(self):
#         """Initialize with same block color mappings"""
#         self.blocks_to_cols = {
#             0: (0.5, 0.25, 0.0),    # light brown
#             10: 'black', # bedrock
#             29: "#006400", # cacutus
#             38: "#B8860B",  # clay
#             60: "brown",  # dirt
#             92: "gold",  # gold ore
#             93: "green",  # grass
#             115: "brown",  # ladder...?
#             119: (.02, .28, .16, 0.9),  # transparent forest green (RGBA) for leaves
#             120: (.02, .28, .16, 0.9),  # leaves2
#             194: "yellow",  # sand
#             217: "gray",  # stone
#             240: (0.0, 0.0, 1.0, 0.4),  # water
#             227: (0.0, 1.0, 0.0, .3), # tall grass
#             237: (0.33, 0.7, 0.33, 0.3), # vine
#             40: "#2F4F4F",  # coal ore
#             62: "#228B22",  # double plant
#             108: "#BEBEBE",  # iron ore
#             131: "saddlebrown",  # log1
#             132: "saddlebrown",  #log2
#             95: "lightgray",  # gravel
#             243: "wheat",  # wheat
#             197: "limegreen",  # sapling
#             166: "orange",  #pumpkin
#             167: "#FF8C00",  # pumpkin stem
#             184: "#FFA07A",  # red flower
#             195: "tan",  # sandstone
#             250: "white",  #wool 
#             251: "gold",   #yellow flower
#         }
#         try:
#             import panel as pn
#             pn.extension('vtk')
#             pv.set_jupyter_backend('trame')
#         except ImportError:
#             print("Please install panel with: pip install panel")
        
#     def visualize_chunk(self, voxels, highlight_latents=None, plotter=None, interactive=False):
#         """Visualize a single chunk with optional latent space highlighting"""
#         # Convert to numpy if needed
#         if isinstance(voxels, torch.Tensor):
#             if voxels.dim() == 4:  # One-hot encoded [C,H,W,D]
#                 voxels = voxels.detach().cpu()
#                 voxels = torch.argmax(voxels, dim=0).numpy()
#             else:
#                 voxels = voxels.detach().cpu().numpy()
                
#         # Apply the same transformations as original
#         voxels = voxels.transpose(2, 0, 1)
#         # Rotate the voxels 90 degrees around the height axis
#         voxels = np.rot90(voxels, 1, (0, 1))
                
#         # Create grid
#         grid = pv.ImageData()
#         grid.dimensions = np.array(voxels.shape) + 1
#         grid.cell_data["values"] = voxels.flatten(order="F")
        
#         # Create plotter if not provided
#         if plotter is None:
#             if interactive:
#                 plotter = pv.Plotter(notebook=True)
#             else:
#                 plotter = pv.Plotter(off_screen=True)
        
#         # Remove existing lights
#         plotter.remove_all_lights()
        
#         # Add the three-point lighting setup
#         plotter.add_light(pv.Light(
#             position=(1, -1, 1),
#             intensity=1.0,
#             color='white'
#         ))
        
#         plotter.add_light(pv.Light(
#             position=(-1, 1, 0.5),
#             intensity=0.5,
#             color='white'
#         ))
        
#         plotter.add_light(pv.Light(
#             position=(-0.5, -0.5, -1),
#             intensity=0.3,
#             color='white'
#         ))
        
#         # Plot each block type
#         mask = (voxels != 5) & (voxels != -1)
#         unique_blocks = np.unique(voxels[mask])
        
#         for block_id in unique_blocks:
#             threshold = grid.threshold([block_id-0.5, block_id+0.5])
#             if block_id in self.blocks_to_cols:
#                 color = self.blocks_to_cols[int(block_id)]
#                 opacity = 1.0 if isinstance(color, str) or len(color) == 3 else color[3]
#             else:
#                 color = (1.0, 0.0, 0.0)
#                 opacity = 0.2
            
#             plotter.add_mesh(threshold, 
#                         color=color,
#                         opacity=opacity,
#                         show_edges=True,
#                         edge_color='black',
#                         line_width=.2,
#                         edge_opacity=0.2,
#                         lighting=True)
            
#         # Add highlight boxes if specified
#         if highlight_latents is not None:
#             coords = np.array(highlight_latents)
#             scale = 4
            
#             # Find the bounds of all coordinates
#             d_coords = coords[:, 0]
#             h_coords = coords[:, 1]
#             w_coords = coords[:, 2]
            
#             # Create bounds for the cube
#             bounds = (
#                 abs(5 - d_coords.max()) * scale, abs(5 - d_coords.min() + 1) * scale,  # X bounds
#                 w_coords.min() * scale, (w_coords.max() + 1) * scale,                  # Y bounds
#                 h_coords.min() * scale, (h_coords.max() + 1) * scale                   # Z bounds
#             )
            
#             # Add transparent faces
#             cube_faces = pv.Cube(bounds=bounds)
#             plotter.add_mesh(cube_faces, 
#                             color='red',
#                             opacity=0.1,
#                             show_edges=False,
#                             lighting=False)
            
#             # Add opaque edges as a separate wireframe
#             cube_edges = pv.Cube(bounds=bounds)
#             plotter.add_mesh(cube_edges,
#                             color='red',
#                             style='wireframe',
#                             line_width=1,
#                             opacity=.5,
#                             lighting=False)
                

#         # Add dummy cube for bounds
#         outline = pv.Cube(bounds=(0, 24, 0, 24, 0, 24))
#         plotter.add_mesh(outline, opacity=0.0)
        
#         # Add bounds with consistent settings
#         plotter.show_bounds(
#             grid='back',
#             location='back',
#             font_size=8,
#             bold=False,
#             font_family='arial',
#             use_2d=False,
#             bounds=[0, 24, 0, 24, 0, 24],
#             axes_ranges=[0, 24, 0, 24, 0, 24],
#             padding=0.0,
#             n_xlabels=2,
#             n_ylabels=2,
#             n_zlabels=2
#         )
        
#         # Set camera position and zoom
#         plotter.camera_position = 'iso'
#         plotter.camera.zoom(1)
        
#         return plotter
    
#     def visualize_interactive(self, voxels):
#         # Convert to numpy if needed
#         if isinstance(voxels, torch.Tensor):
#             if voxels.dim() == 4:  # One-hot encoded [C,H,W,D]
#                 voxels = voxels.detach().cpu()
#                 voxels = torch.argmax(voxels, dim=0).numpy()
#             else:
#                 voxels = voxels.detach().cpu().numpy()
                
#         # Apply the same transformations as original
#         voxels = voxels.transpose(2, 0, 1)
#         # Rotate the voxels 90 degrees around the height axis
#         voxels = np.rot90(voxels, 1, (0, 1))
                
#         # Create grid
#         grid = pv.ImageData()
#         grid.dimensions = np.array(voxels.shape) + 1
#         grid.cell_data["values"] = voxels.flatten(order="F")
        
#         # Create plotter
#         plotter = pv.Plotter(notebook=True)
        
#         # Remove existing lights
#         plotter.remove_all_lights()
        
#         # Add custom lights
#         # Main light from top-front-right (sun-like)
#         # Add a headlight (light from camera position)
#         # Key light (main light, 45 degrees from front-right)
#         plotter.add_light(pv.Light(
#             position=(1, -1, 1),
#             intensity=1.0,
#             color='white'
#         ))
        
#         # Fill light (softer light from opposite side)
#         plotter.add_light(pv.Light(
#             position=(-1, 1, 0.5),
#             intensity=0.5,
#             color='white'
#         ))
        
#         # Back light (rim lighting from behind)
#         plotter.add_light(pv.Light(
#             position=(-0.5, -0.5, -1),
#             intensity=0.3,
#             color='white'
#     ))
        
#         # Plot each block type
#         mask = (voxels != 5) & (voxels != -1)
#         unique_blocks = np.unique(voxels[mask])
        
#         for block_id in unique_blocks:
#             threshold = grid.threshold([block_id-0.5, block_id+0.5])
#             if block_id in self.blocks_to_cols:
#                 color = self.blocks_to_cols[int(block_id)]
#                 opacity = 1.0 if isinstance(color, str) or len(color) == 3 else color[3]
#             else:
#                 color = (1.0, 0.0, 0.0)
#                 opacity = 0.2
            
#             plotter.add_mesh(threshold, 
#                         color=color,
#                         opacity=opacity,
#                         show_edges=True,
#                         edge_color='black',
#                         line_width=.2,   # Thin edges
#                         edge_opacity=0.2,
#                         lighting=True)
        
#         # Add a dummy cube to force the bounds
#         outline = pv.Cube(bounds=(0, 24, 0, 24, 0, 24))
#         plotter.add_mesh(outline, opacity=0.0)  # Invisible cube to set bounds
        
#         # Add clean axes with consistent range
#         plotter.show_bounds(
#             grid='back',
#             location='back',
#             # all_edges=True,
#             # ticks=None,
#             font_size=8,
#             bold=False,
#             font_family='arial',
#             use_2d=False,
#             bounds=[0, 24, 0, 24, 0, 24],
#             axes_ranges=[0, 24, 0, 24, 0, 24],
#             padding=0.0,
#             n_xlabels=2,
#             n_ylabels=2,
#             n_zlabels=2,
#             # show_xlabels=False,
#             # show_ylabels=False,
#             # show_zlabels=False
#         )
        
#         # Set camera position and zoom
#         plotter.camera_position = 'iso'
#         plotter.camera.zoom(1)
        
#         return plotter
    
#     def visualize_latent_space(self, latents, latent_type="style", plotter=None):
#         """
#         Interactive visualization of latent codes in their spatial positions.
#         """
#         # Handle batch dimension if present
#         if isinstance(latents, torch.Tensor):
#             if latents.dim() == 4:  # [1,6,6,6]
#                 latents = latents.squeeze(0)
#             latents = latents.detach().cpu().numpy()
        
#         if plotter is None:
#             plotter = pv.Plotter(notebook=True)
        
#         # Add title to the plot
#         plotter.add_title(f"{latent_type.capitalize()} Latent Codes", font_size=16)
        
#         # Store cubes and text separately
#         cubes_by_layer = {i: [] for i in range(6)}
#         text_actors = {}  # Store one text actor per layer
        
#         # Create colormap
#         import matplotlib.cm as cm
#         max_code = np.max(latents)
#         min_code = np.min(latents)
#         norm = plt.Normalize(min_code, max_code)
#         cmap = cm.viridis  # You can try other colormaps like 'plasma', 'magma', etc.

#         # Create all cubes and text for all layers
#         for layer in range(6):  # This is the height (H)
#             # Create all text for this layer at once
#             centers = []
#             labels = []
#             for d in range(6):  # This is depth (D)
#                 for w in range(6):  # This is width (W)
#                     # Transform coordinates for visualization while keeping original indices for values
#                     x = abs(5 - d)  # Transform depth to x-coordinate
#                     # Get color for this latent code
#                     latent_value = latents[d, layer, w]
#                     rgba_color = cmap(norm(latent_value))

#                     cube = pv.Cube(
#                         bounds=(
#                             x * 4, (x + 1) * 4,        # X (transformed from depth)
#                             w * 4, (w + 1) * 4,        # Y (width)
#                             layer * 4, (layer + 1) * 4  # Z (height)
#                         )
#                     )
                    
#                     # Add cube
#                     actor = plotter.add_mesh(cube,
#                                         color=rgba_color[:3],
#                                         opacity=0.1,
#                                         show_edges=True,
#                                         edge_color='blue',
#                                         line_width=2)
#                     cubes_by_layer[layer].append(actor)
                    
#                     # Collect center and label for text
#                     centers.append(cube.center)
#                     labels.append(f"{latents[d, layer, w]}")  # [D,H,W] ordering
            
#             # Create text actor for this layer
#             text_actor = plotter.add_point_labels(
#                 centers,
#                 labels,
#                 font_size=16,
#                 always_visible=True,
#                 shape_opacity=0.0,
#                 text_color='black'
#             )
#             text_actors[layer] = text_actor
            
#             # Hide all layers except first
#             if layer != 0:
#                 for cube in cubes_by_layer[layer]:
#                     cube.visibility = False
#                 plotter.remove_actor(text_actor)
        
#         def update_layer(value):
#             layer = int(value)
#             # Update cube visibility
#             for l in range(6):
#                 for cube in cubes_by_layer[l]:
#                     cube.visibility = (l == layer)
                
#                 # Update text visibility
#                 if l == layer:
#                     plotter.add_actor(text_actors[l])
#                 else:
#                     plotter.remove_actor(text_actors[l])
            
#             plotter.render()
        
#         # Add slider widget
#         slider = plotter.add_slider_widget(
#             update_layer,
#             [0, 5],
#             value=0,
#             title='Layer',
#             pointa=(0.025, 0.1),
#             pointb=(0.225, 0.1),
#             style='modern',
#             fmt='%0.0f'
#         )
        
#         # Set initial camera position
#         plotter.camera_position = 'iso'
#         plotter.camera.zoom(1.2)
        
#         return plotter
    
#     def create_voxel_gif(self, voxel_sequence, output_path='voxel_animation.gif', duration=2.0, highlight_latents=None):
#         """
#         Create a GIF animation from a sequence of voxel arrays.
        
#         Args:
#             voxel_sequence: List of voxel arrays (same format as visualize_chunk input)
#             output_path: Path to save the output GIF
#             duration: Total duration of the GIF in seconds
#         """
#         import imageio.v2 as imageio
        
#         # Calculate frame duration (in seconds)
#         frame_duration = duration / len(voxel_sequence)
#         print(f"Total frames: {len(voxel_sequence)}")
#         print(f"Frame duration: {frame_duration} seconds")
        
#         # Create a plotter
#         plotter = pv.Plotter(off_screen=True)
        
#         # Store frames
#         frames = []
        
#         # Render each voxel array
#         for voxels in voxel_sequence:
#             # Clear previous frame
#             plotter.clear()
            
#             # Visualize the current voxels
#             if highlight_latents is not None:
#                 plotter = self.visualize_chunk(voxels, plotter=plotter, highlight_latents=highlight_latents)
#             else:
#                 plotter = self.visualize_chunk(voxels, plotter=plotter)
            
#             # Capture the frame
#             frame = plotter.screenshot(transparent_background=False, return_img=True)
#             frames.append(frame)
        
#         # Save as GIF
#         # imageio.mimwrite(output_path, frames, duration=duration, loop=0)
#         imageio.mimwrite(output_path, frames, fps = 2, loop=0)

        
#         return output_path
    
#     def visualize_latent_space_with_blocks(self, latents, block_data, latent_type="style", plotter=None):
#         """
#         Interactive visualization of latent codes next to minecraft blocks.
        
#         Args:
#             latents: Tensor of latent indices [6,6,6] or [1,6,6,6]
#             block_data: Tensor of minecraft blocks
#             latent_type: "style" or "struct" for color scheme
#             plotter: Optional existing plotter
#         """
#         if plotter is None:
#             plotter = pv.Plotter(notebook=True)
        
#         # Handle batch dimension if present
#         if isinstance(latents, torch.Tensor):
#             if latents.dim() == 4:
#                 latents = latents.squeeze(0)
#             latents = latents.detach().cpu().numpy()
        
#         # Add title to the plot
#         plotter.add_title(f"{latent_type.capitalize()} Latent Codes", font_size=16)
        
#         # Store cubes and text separately
#         cubes_by_layer = {i: [] for i in range(6)}
#         text_actors = {}
        
#         # Create colormap
#         import matplotlib.cm as cm
#         max_code = np.max(latents)
#         min_code = np.min(latents)
#         norm = plt.Normalize(min_code, max_code)
#         cmap = cm.viridis
        
#         # Create all cubes and text for latent visualization (offset in x direction)
#         offset = 24  # Space between visualizations
#         for layer in range(6):
#             centers = []
#             labels = []
#             for d in range(6):
#                 for w in range(6):
#                     x = abs(5 - d)
#                     latent_value = latents[d, layer, w]
#                     rgba_color = cmap(norm(latent_value))
                    
#                     cube = pv.Cube(bounds=(
#                         x * 4 + offset, (x + 1) * 4 + offset,  # Offset in x direction
#                         w * 4, (w + 1) * 4,
#                         layer * 4, (layer + 1) * 4
#                     ))
                    
#                     actor = plotter.add_mesh(cube,
#                                         color=rgba_color[:3],
#                                         opacity=0.8,
#                                         show_edges=True,
#                                         edge_color='black',
#                                         line_width=1)
#                     cubes_by_layer[layer].append(actor)
                    
#                     centers.append(cube.center)
#                     labels.append(f"{latents[d, layer, w]}")
            
#             text_actor = plotter.add_point_labels(
#                 centers,
#                 labels,
#                 font_size=16,
#                 always_visible=True,
#                 shape_opacity=0.0,
#                 text_color='black'
#             )
#             text_actors[layer] = text_actor
            
#             if layer != 0:
#                 for cube in cubes_by_layer[layer]:
#                     cube.visibility = False
#                 plotter.remove_actor(text_actor)
        
#         # Add minecraft visualization
#         self.visualize_chunk(block_data, plotter=plotter)
        
#         def update_layer(value):
#             layer = int(value)
#             for l in range(6):
#                 for cube in cubes_by_layer[l]:
#                     cube.visibility = (l == layer)
                
#                 if l == layer:
#                     plotter.add_actor(text_actors[l])
#                 else:
#                     plotter.remove_actor(text_actors[l])
            
#             plotter.render()
        
#         # Add slider widget
#         slider = plotter.add_slider_widget(
#             update_layer,
#             [0, 5],
#             value=0,
#             title='Layer',
#             pointa=(0.025, 0.1),
#             pointb=(0.225, 0.1),
#             style='modern',
#             fmt='%0.0f'
#         )
        
#         # # Add colorbar
#         # plotter.add_scalar_bar(title=f'{latent_type.capitalize()} Latent Values',
#         #                     n_labels=5,
#         #                     mapper=plt.cm.ScalarMappable(norm=norm, cmap=cmap))
        
#         # Set initial camera position
#         plotter.camera_position = 'iso'
#         plotter.camera.zoom(0.8)  # Zoom out a bit to see both visualizations
        
#         return plotter
    
#     def visualize_both_codes_with_blocks(self, style_latents, struct_latents, block_data, plotter=None):
#         """
#         More efficient visualization of style and structure codes next to minecraft blocks.
#         Uses pre-computed actors and optimized updates.
#         """
#         if plotter is None:
#             plotter = pv.Plotter(notebook=False)
        
#         # Handle batch dimensions and convert to numpy
#         if isinstance(style_latents, torch.Tensor):
#             if style_latents.dim() == 4:
#                 style_latents = style_latents.squeeze(0)
#             style_latents = style_latents.detach().cpu().numpy()
#         if isinstance(struct_latents, torch.Tensor):
#             if struct_latents.dim() == 4:
#                 struct_latents = struct_latents.squeeze(0)
#             struct_latents = struct_latents.detach().cpu().numpy()
        
#         # Add title
#         plotter.add_title("Style and Structure Latent Codes", font_size=16)
        
#         # Create colormaps
#         import matplotlib.cm as cm
#         style_norm = plt.Normalize(np.min(style_latents), np.max(style_latents))
#         struct_norm = plt.Normalize(np.min(struct_latents), np.max(struct_latents))
#         style_cmap = cm.viridis
#         struct_cmap = cm.plasma
        
#         # Pre-compute all actors for each layer
#         style_actors = {i: [] for i in range(6)}
#         struct_actors = {i: [] for i in range(6)}
#         style_texts = {}
#         struct_texts = {}
        
#         # Create all actors at once
#         for layer in range(6):
#             style_centers = []
#             struct_centers = []
#             style_labels = []
#             struct_labels = []
            
#             for d in range(6):
#                 for w in range(6):
#                     x = abs(5 - d)
                    
#                     # Style latents (left side)
#                     style_value = style_latents[d, layer, w]
#                     style_color = style_cmap(style_norm(style_value))
#                     style_cube = pv.Cube(bounds=(
#                         x * 4 + 24, (x + 1) * 4 + 24,
#                         w * 4, (w + 1) * 4,
#                         layer * 4, (layer + 1) * 4
#                     ))
#                     style_actor = plotter.add_mesh(
#                         style_cube,
#                         color=style_color[:3],
#                         opacity=0.8,
#                         show_edges=True,
#                         edge_color='black',
#                         line_width=1
#                     )
#                     style_actors[layer].append(style_actor)
#                     style_centers.append(style_cube.center)
#                     style_labels.append(f"{style_value}")
                    
#                     # Structure latents (right side)
#                     struct_value = struct_latents[d, layer, w]
#                     struct_color = struct_cmap(struct_norm(struct_value))
#                     struct_cube = pv.Cube(bounds=(
#                         x * 4, (x + 1) * 4,
#                         w * 4 + 24, (w + 1) * 4 + 24,
#                         layer * 4, (layer + 1) * 4
#                     ))
#                     struct_actor = plotter.add_mesh(
#                         struct_cube,
#                         color=struct_color[:3],
#                         opacity=0.8,
#                         show_edges=True,
#                         edge_color='black',
#                         line_width=1
#                     )
#                     struct_actors[layer].append(struct_actor)
#                     struct_centers.append(struct_cube.center)
#                     struct_labels.append(f"{struct_value}")
            
#             # Create text actors for each layer
#             style_text = plotter.add_point_labels(
#                 style_centers,
#                 style_labels,
#                 font_size=16,
#                 always_visible=True,
#                 shape_opacity=0.0,
#                 text_color='black'
#             )
#             struct_text = plotter.add_point_labels(
#                 struct_centers,
#                 struct_labels,
#                 font_size=16,
#                 always_visible=True,
#                 shape_opacity=0.0,
#                 text_color='black'
#             )
            
#             style_texts[layer] = style_text
#             struct_texts[layer] = struct_text
            
#             # Hide all layers except first
#             if layer != 0:
#                 for actor in style_actors[layer]:
#                     actor.visibility = False
#                 for actor in struct_actors[layer]:
#                     actor.visibility = False
#                 plotter.remove_actor(style_text)
#                 plotter.remove_actor(struct_text)
        
#         # Add static labels
#         for label, pos, rot in [
#             ("Style codes", (50, 12, 0), 90),
#             ("Structure codes", (12, 50, 0), 180)
#         ]:
#             text = pv.Text3D(label, depth=0.3, height=2)
#             text = text.rotate_z(rot)
#             text = text.translate(pos)
#             plotter.add_mesh(text, color='black')
        
#         # Add minecraft visualization
#         self.visualize_chunk(block_data, plotter=plotter)
        
#         def update_layer(value):
#             layer = int(value)
#             # Use batch operations where possible
#             for l in range(6):
#                 is_visible = (l == layer)
#                 # Update cube visibility
#                 for actor in style_actors[l]:
#                     actor.visibility = is_visible
#                 for actor in struct_actors[l]:
#                     actor.visibility = is_visible
                
#                 # Update text visibility
#                 if is_visible:
#                     plotter.add_actor(style_texts[l])
#                     plotter.add_actor(struct_texts[l])
#                 else:
#                     plotter.remove_actor(style_texts[l])
#                     plotter.remove_actor(struct_texts[l])
            
#             plotter.render()
        
#         # Add slider
#         plotter.add_slider_widget(
#             update_layer,
#             [0, 5],
#             value=0,
#             title='Layer',
#             pointa=(0.025, 0.1),
#             pointb=(0.225, 0.1),
#             style='modern',
#             fmt='%0.0f'
#         )
        
#         # Set camera
#         plotter.camera_position = 'iso'
#         plotter.camera.zoom(0.7)
        
#         return plotter

# Block Converter

In [4]:
# class BlockBiomeConverter:
#     def __init__(self, block_mappings=None, biome_mappings=None):
#         """
#         Initialize with pre-computed mappings for both blocks and biomes
        
#         Args:
#             block_mappings: dict containing 'index_to_block' and 'block_to_index'
#             biome_mappings: dict containing 'index_to_biome' and 'biome_to_index'
#         """
#         self.index_to_block = block_mappings['index_to_block'] if block_mappings else None
#         self.block_to_index = block_mappings['block_to_index'] if block_mappings else None
#         self.index_to_biome = biome_mappings['index_to_biome'] if biome_mappings else None
#         self.biome_to_index = biome_mappings['biome_to_index'] if biome_mappings else None
    
#     @classmethod
#     def from_dataset(cls, data_path):
#         """Create mappings from a dataset file"""
#         data = np.load(data_path, allow_pickle=True)
#         voxels = data['voxels']
#         biomes = data['biomes']
        
#         # Create block mappings (blocks are integers)
#         unique_blocks = np.unique(voxels)
#         block_to_index = {int(block): idx for idx, block in enumerate(unique_blocks)}
#         index_to_block = {idx: int(block) for idx, block in enumerate(unique_blocks)}
        
#         # Create biome mappings (biomes are strings)
#         unique_biomes = np.unique(biomes)
#         biome_to_index = {str(biome): idx for idx, biome in enumerate(unique_biomes)}
#         index_to_biome = {idx: str(biome) for idx, biome in enumerate(unique_biomes)}
        
#         block_mappings = {'index_to_block': index_to_block, 'block_to_index': block_to_index}
#         biome_mappings = {'index_to_biome': index_to_biome, 'biome_to_index': biome_to_index}
        
#         return cls(block_mappings, biome_mappings)
    
#     @classmethod
#     def from_arrays(cls, voxels, biomes):
#         """Create mappings directly from numpy arrays"""
#         # Create block mappings (blocks are integers)
#         unique_blocks = np.unique(voxels)
#         block_to_index = {int(block): idx for idx, block in enumerate(unique_blocks)}
#         index_to_block = {idx: int(block) for idx, block in enumerate(unique_blocks)}
        
#         # Create biome mappings (biomes are strings)
#         unique_biomes = np.unique(biomes)
#         biome_to_index = {str(biome): idx for idx, biome in enumerate(unique_biomes)}
#         index_to_biome = {idx: str(biome) for idx, biome in enumerate(unique_biomes)}
        
#         block_mappings = {'index_to_block': index_to_block, 'block_to_index': block_to_index}
#         biome_mappings = {'index_to_biome': index_to_biome, 'biome_to_index': biome_to_index}
        
#         return cls(block_mappings, biome_mappings)
    
#     @classmethod
#     def load_mappings(cls, path):
#         """Load pre-saved mappings"""
#         mappings = torch.load(path)
#         return cls(mappings['block_mappings'], mappings['biome_mappings'])
    
#     def save_mappings(self, path):
#         """Save mappings for later use"""
#         torch.save({
#             'block_mappings': {
#                 'index_to_block': self.index_to_block,
#                 'block_to_index': self.block_to_index
#             },
#             'biome_mappings': {
#                 'index_to_biome': self.index_to_biome,
#                 'biome_to_index': self.biome_to_index
#             }
#         }, path)
    
#     def convert_to_original_blocks(self, data):
#         """
#         Convert from indices back to original block IDs.
#         Handles both one-hot encoded and already-indexed data.
        
#         Args:
#             data: torch.Tensor of either:
#                 - one-hot encoded blocks [B, C, H, W, D] or [C, H, W, D]
#                 - indexed blocks [B, H, W, D] or [H, W, D]
#         Returns:
#             torch.Tensor of original block IDs with shape [B, H, W, D] or [H, W, D]
#         """
#         # If one-hot encoded (dim == 5 or first dim == num_blocks), convert to indices first
#         if len(data.shape) == 5 or (len(data.shape) == 4 and data.shape[0] == len(self.block_to_index)):
#             data = torch.argmax(data, dim=1 if len(data.shape) == 5 else 0)
        
#         # Now convert indices to original blocks
#         if len(data.shape) == 4:  # Batch dimension present
#             return torch.tensor([[[[self.index_to_block[int(b)] 
#                                 for b in row]
#                                 for row in layer]
#                                 for layer in slice_]
#                                 for slice_ in data])
#         else:  # No batch dimension
#             return torch.tensor([[[self.index_to_block[int(b)] 
#                                 for b in row]
#                                 for row in layer]
#                                 for layer in data])

#     def convert_to_original_biomes(self, data):
#         """
#         Convert from indices back to original biome strings.
#         Handles both one-hot encoded and already-indexed data.
        
#         Args:
#             data: torch.Tensor of either:
#                 - one-hot encoded biomes [B, C, H, W, D] or [C, H, W, D]
#                 - indexed biomes [B, H, W, D] or [H, W, D]
#         Returns:
#             numpy array of original biome strings with shape [B, H, W, D] or [H, W, D]
#         """
#         # If one-hot encoded (dim == 5 or first dim == num_biomes), convert to indices first
#         if len(data.shape) == 5 or (len(data.shape) == 4 and data.shape[0] == len(self.biome_to_index)):
#             data = torch.argmax(data, dim=1 if len(data.shape) == 5 else 0)
        
#         # Now convert indices to original biomes
#         if len(data.shape) == 4:  # Batch dimension present
#             return np.array([[[[self.index_to_biome[int(b)] 
#                             for b in row]
#                             for row in layer]
#                             for layer in slice_]
#                             for slice_ in data])
#         else:  # No batch dimension
#             return np.array([[[self.index_to_biome[int(b)] 
#                             for b in row]
#                             for row in layer]
#                             for layer in data])

# Minecraft Chunks Dataset

In [5]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

class MinecraftDataset(Dataset):
    def __init__(self, data_path):
        data_path = Path(data_path)

        # Try to load processed data first
        # assert processed_data_path.exists() and mappings_path.exists()

        print("Loading pre-processed data...")
        processed_data = torch.load(data_path)
        # Only keep the chunks, discard biome data
        self.processed_chunks = processed_data['chunks']
        # Delete the biomes to free memory
        del processed_data['biomes']
        del processed_data
        
        
        print(f"Loaded {len(self.processed_chunks)} chunks of size {self.processed_chunks.shape[1:]}")
        print(f"Number of unique block types: {self.processed_chunks.shape[1]}")
        print(f'Unique blocks: {torch.unique(torch.argmax(self.processed_chunks, dim=1)).tolist()}')

    def __getitem__(self, idx):
        return self.processed_chunks[idx]

    def __len__(self):
        return len(self.processed_chunks)

def get_minecraft_dataloaders(data_path, batch_size=32, val_split=0.1, num_workers=4):
    """
    Creates training and validation dataloaders for Minecraft chunks.
    """
    # Create dataset
    dataset = MinecraftDataset(data_path)
    
    # Split into train and validation sets
    val_size = int(val_split * len(dataset))
    train_size = len(dataset) - val_size
    
    # Use a fixed seed for reproducibility
    generator = torch.Generator().manual_seed(42)
    
    train_dataset, val_dataset = torch.utils.data.random_split(
        dataset, 
        [train_size, val_size],
        generator=generator
    )
    
    # Create dataloaders with memory pinning
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True,
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True,
    )

    
    print(f"\nDataloader details:")
    print(f"Training samples: {len(train_dataset)}")
    print(f"Validation samples: {len(val_dataset)}")
    print(f"Batch size: {batch_size}")
    print(f"Training batches: {len(train_loader)}")
    print(f"Validation batches: {len(val_loader)}")
    
    return train_loader, val_loader


# Helper Functions
Loads hyperparameters and model from checkpoint, gives easy function to encode a structure into latent codes, as well as decode those latents back into the reconstruction

## VQGAN Helper Functions

In [6]:
import os
from log_utils import log, load_stats, load_model
import copy
from hyperparams import HparamsVQGAN

# Loads hparams from hparams.json file in saved model directory
def vq_load_hparams_from_json(log_dir):
    import json
    import os
    json_path = os.path.join(log_dir, 'hparams.json')
    
    if not os.path.exists(json_path):
        raise FileNotFoundError(f"No hparams.json file found in {log_dir}")
    
    with open(json_path, 'r') as f:
        hparams = json.load(f)

    return hparams

# turns loaded hparams json into propery hyperparams object
def vq_dict_to_vcqgan_hparams(hparams_dict, dataset=None):
    # Determine which hyperparameter class to use based on the dataset
    if dataset == None:
        dataset = hparams_dict.get('dataset', 'MNIST')  # Default to MNIST if not specified
    
    vq_hyper = HparamsVQGAN(dataset)
    # Set attributes from the dictionary
    for key, value in hparams_dict.items():
        setattr(vq_hyper, key, value)
    
    return vq_hyper

def load_model_vq(model, model_load_name, step, log_dir, strict=False):
    checkpoint_path = os.path.join("../model_logs", log_dir, "saved_models", f"{model_load_name}_{step}.th")
    print(f"\nAttempting to load checkpoint from: {checkpoint_path}")
    
    if not os.path.exists(checkpoint_path):
        raise FileNotFoundError(f"Checkpoint not found at: {checkpoint_path}")
        
    # Load checkpoint
    checkpoint = torch.load(checkpoint_path)
    
    # Create new state dict with modified keys
    new_state_dict = {}
    for key, value in checkpoint.items():
        if key.startswith('ae.'):
            # Remove the 'ae.' prefix
            new_key = key[3:]  # Skip 'ae.'
            new_state_dict[new_key] = value
    
    # Try loading state dict
    try:
        model.load_state_dict(new_state_dict, strict=strict)
        
        print("\nLoad successful!")
    except Exception as e:
        print(f"\nError loading state dict: {e}")
        
    return model
def vq_load_vqgan_from_checkpoint(H, vqgan):
    vqgan = load_model_vq(vqgan, "vqgan", H.load_step, H.load_dir).cuda()
    vqgan.eval()
    return vqgan


def vq_encode_and_quantize(vqgan, terrain_chunks, device='cuda'):
    """Memory-efficient encoding function for single codebook VQGAN"""
    vqgan.eval()
    with torch.no_grad():
        # Move input to device and ensure it's one-hot encoded
        terrain_chunks = terrain_chunks.to(device)
        
        # Get encodings through encoder
        latents = vqgan.encoder(terrain_chunks)
        
        # Get quantized indices from VQ layer
        _, _, quant_stats = vqgan.quantize(latents)
        indices = quant_stats["min_encoding_indices"]
        
        # Move to CPU and clear GPU memory
        indices = indices.cpu()
        torch.cuda.empty_cache()
        
        return indices

def vq_decode_from_indices(indices, vqgan, device='cuda'):
    """Reconstructs from latent indices"""
    with torch.no_grad():
        # Move indices to device
        indices = indices.to(device)
        
        # Get original shape for reshaping
        batch_size = indices.size(0)
        
        # Convert indices to quantized embeddings using the VQ layer's get_codebook_entry
        shape = (batch_size, 6, 6, 6, vqgan.embed_dim)  # Shape for 3D data
        quant = vqgan.quantize.get_codebook_entry(indices.reshape(-1), shape)
        
        # Generate output through decoder
        decoded = vqgan.generator(quant)
        
        # Convert to block IDs if one-hot encoded
        if decoded.shape[1] > 1:
            decoded = torch.argmax(decoded, dim=1)
        
        # Move result to CPU and clear GPU memory
        decoded = decoded.squeeze(0).cpu()
        torch.cuda.empty_cache()
        
        return decoded

## FQGAN helper functions

In [7]:
import os
from log_utils import log, load_stats, load_model
import copy
from fq_models import FQModel, HparamsFQGAN


# Loads hparams from hparams.json file in saved model directory
def load_hparams_from_json(log_dir):
    import json
    import os
    json_path = os.path.join(log_dir, 'hparams.json')
    
    if not os.path.exists(json_path):
        raise FileNotFoundError(f"No hparams.json file found in {log_dir}")
    
    with open(json_path, 'r') as f:
        hparams = json.load(f)

    return hparams

# turns loaded hparams json into propery hyperparams object
def dict_to_vcqgan_hparams(hparams_dict, dataset=None):
    # Determine which hyperparameter class to use based on the dataset
    if dataset == None:
        dataset = hparams_dict.get('dataset', 'MNIST')  # Default to MNIST if not specified
    
    vq_hyper = HparamsFQGAN(dataset)
    # Set attributes from the dictionary
    for key, value in hparams_dict.items():
        setattr(vq_hyper, key, value)
    
    return vq_hyper

# Loads fqgan model weights from a given checkpoint file
def load_fqgan_from_checkpoint(H, fqgan):
    fqgan = load_model(fqgan, "fqgan", H.load_step, H.load_dir).cuda()
    fqgan.eval()
    return fqgan

# Takes a chunk or batch of chunks from the dataset, returns the encoded style and structure indices matrices
def encode_and_quantize(fqgan, terrain_chunks, device='cuda'):
    """Memory-efficient encoding function"""
    fqgan.eval()
    with torch.no_grad():
        # Move input to device
        terrain_chunks = terrain_chunks.to(device)
        
        # Get encodings
        h_style, h_struct = fqgan.encoder(terrain_chunks)
        
        # Process style path
        h_style = fqgan.quant_conv_style(h_style)
        quant_style, _, style_stats = fqgan.quantize_style(h_style)
        style_indices = style_stats[2]  # Get indices from tuple
        style_indices = style_indices.view(
            (h_style.size()[0], h_style.size()[2], h_style.size()[3], h_style.size()[4])
        )
        
        # Clear intermediate tensors
        del h_style, quant_style, style_stats
        
        # Process structure path
        h_struct = fqgan.quant_conv_struct(h_struct)
        quant_struct, _, struct_stats = fqgan.quantize_struct(h_struct)
        struct_indices = struct_stats[2]  # Get indices from tuple
        struct_indices = struct_indices.view(
            (h_struct.size()[0], h_struct.size()[2], h_struct.size()[3], h_struct.size()[4])
        )
        
        # Clear intermediate tensors
        del h_struct, quant_struct, struct_stats
        
        # Move indices to CPU to save GPU memory
        style_indices = style_indices.cpu()
        struct_indices = struct_indices.cpu()
        
        torch.cuda.empty_cache()
        
        return style_indices, struct_indices

# Takes style and structure indices, returns the reconstructed map
def decode_from_indices(style_indices, struct_indices, fqgan, device='cuda', two_stage=False, return_raw=False):
    """Memory-efficient decoding function"""
    with torch.no_grad():
        # Move indices to device only when needed
        style_indices = style_indices.to(device)
        struct_indices = struct_indices.to(device)
        
        # Get quantized vectors
        quant_style = fqgan.quantize_style.get_codebook_entry(
            style_indices.view(-1),
            shape=[1, fqgan.embed_dim, *style_indices.shape[1:]]
        )
        quant_struct = fqgan.quantize_struct.get_codebook_entry(
            struct_indices.view(-1),
            shape=[1, fqgan.embed_dim, *struct_indices.shape[1:]]
        )
        
        # Clear indices from GPU
        del style_indices, struct_indices
        
        # Combine and decode
        quant = torch.cat([quant_struct, quant_style], dim=1)
        # quant = quant_style + quant_struct
        del quant_style, quant_struct
        
        if two_stage:
            decoded, binary_decoded = fqgan.decoder(quant)
        else:
            decoded = fqgan.decoder(quant)
        
        del quant

        if return_raw:
            return decoded
        
        # Convert to block IDs if one-hot encoded
        if decoded.shape[1] > 1:
            decoded = torch.argmax(decoded, dim=1)
        
        # Move result to CPU and clear GPU memory
        result = decoded.squeeze(0).cpu()
        if two_stage:
            binary_result = binary_decoded.squeeze(0).cpu()
            del decoded
            del binary_decoded
            torch.cuda.empty_cache()
            return result, binary_result
        
        del decoded
        torch.cuda.empty_cache()
        
        return result

# Load FQ model

In [8]:
# Important if you reload the model, can run into memory issues. There's a memory leak somewhere I haven't been able to fix
import gc
gc.collect()
torch.cuda.empty_cache()

In [9]:
# replace with directory for whatever model you want
model_path = '../model_logs/FQGAN_2stagedecoder_logweighted3_32codes_cycleconsistency_STE_postquant_detachcycle_gumbel'

In [26]:
# I'm manually setting the load step here, if it errors out take a look at what the actual .th file number is
fqgan_hparams =  dict_to_vcqgan_hparams(load_hparams_from_json(f"{model_path}"), 'minecraft')
fqgan_hparams.load_step = 5000
# fqgan_hparams.padding_mode = 'reflect'
fqgan = FQModel(fqgan_hparams)
fqgan = load_fqgan_from_checkpoint(fqgan_hparams, fqgan)
print(f'loaded from: {fqgan_hparams.log_dir}')

using padding mode: reflect
With cycle consistency: True type: post_quant_conv, using gumbel: True
Using EMA quantizer
Using SlightlyLessDumbTwoStageGenerator
Detaching binary reconstruction from comp graph for final loss
NO biome supervision
Disentangle Ratio:  0.5
Loading fqgan_5000.th
loaded from: FQGAN_2stagedecoder_logweighted3_32codes_cycleconsistency_STE_postquant_detachcycle_gumbel


# Load VQ Model

In [11]:
# from models3d import VQAutoEncoder

# # vqmodel_path = 'saved_models/minecraft39ch_ce_3'
# vqgan_hparams =  vq_dict_to_vcqgan_hparams(vq_load_hparams_from_json(f"../model_logs/minecraft39ch_ce_3_fqdataset"), 'minecraft')
# vqgan_hparams.load_step = 14999

# vqgan = VQAutoEncoder(vqgan_hparams)
# vqgan = vq_load_vqgan_from_checkpoint(vqgan_hparams, vqgan)
# print(f'loaded from: {vqgan_hparams.log_dir}')

# Load dataset

In [12]:
# loads a preprocesed dataset, which already has a mappings file created and everything one-hot encoded nicely. For more memory efficient, could try just loading the validation set file
data_path = '../../text2env/data/24_newdataset_processed_cleaned3.pt'
train_loader, val_loader = get_minecraft_dataloaders(
    data_path,
    batch_size=4,
    num_workers=0, # Must be 0 if you're on windows, otherwise it errors
    val_split=0.1
)


Loading pre-processed data...
Loaded 11119 chunks of size torch.Size([42, 24, 24, 24])
Number of unique block types: 42
Unique blocks: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41]

Dataloader details:
Training samples: 10008
Validation samples: 1111
Batch size: 4
Training batches: 2502
Validation batches: 278


In [13]:
from data_utils import BlockBiomeConverter
mappings_path = '../../text2env/data/24_newdataset_mappings3.pt'
block_converter = BlockBiomeConverter.load_mappings(mappings_path)

In [15]:
# Sort by index for a cleaner output
print("INDEX | BLOCK ID | BLOCK NAME")
print("-" * 50)
for index in sorted(block_converter.index_to_block.keys()):
    block_id = block_converter.get_block_id_from_index(index)
    block_name = block_converter.get_block_name_from_index(index)
    print(f"{index:5d} | {block_id:7d} | {block_name}")

print(f"\nTotal unique blocks: {len(block_converter.index_to_block)}")

INDEX | BLOCK ID | BLOCK NAME
--------------------------------------------------
    0 |       5 | AIR
    1 |      20 | BONE_BLOCK
    2 |      26 | BROWN_MUSHROOM
    3 |      27 | BROWN_MUSHROOM_BLOCK
    4 |      29 | CACTUS
    5 |      35 | CHEST
    6 |      38 | CLAY
    7 |      40 | COAL_ORE
    8 |      41 | COBBLESTONE
    9 |      56 | DEADBUSH
   10 |      60 | DIRT
   11 |      62 | DOUBLE_PLANT
   12 |      69 | EMERALD_ORE
   13 |      83 | FLOWING_LAVA
   14 |      84 | FLOWING_WATER
   15 |      92 | GOLD_ORE
   16 |      93 | GRASS
   17 |      95 | GRAVEL
   18 |     108 | IRON_ORE
   19 |     117 | LAPIS_ORE
   20 |     118 | LAVA
   21 |     119 | LEAVES
   22 |     120 | LEAVES2
   23 |     131 | LOG
   24 |     132 | LOG2
   25 |     138 | MOB_SPAWNER
   26 |     139 | MONSTER_EGG
   27 |     140 | MOSSY_COBBLESTONE
   28 |     166 | PUMPKIN
   29 |     184 | RED_FLOWER
   30 |     187 | RED_MUSHROOM_BLOCK
   31 |     192 | REEDS
   32 |     194 | SAND
   33 | 

Histograms:
Code 0: Mostly Sandstone, dirt, some: dirt, grass
Code 1: Mostly Air, water
Code 2: Air, water, some: stone
Code 3: Stone
Code 4: Mostly stone, some air
Code 5: Mostly air, some: dirt, grass, stone, water
Code 6: stone
code 7: Air
Code 8: Mostly air, some: dirt, grass, sand, sandstone, stone, water
Code 9: Mostly Stone, some: air, dirt, grass
Code 10: Mostly air, some: dirt, grass, stone, water
Code 11: air
Code 12: Mostly air, some: leaves, stone, water, grass, sand, dirt, gravel, vine
Code 13: WAter/air
Code 14: Mostly air, some stone, little dirt
Code 15: Mostly dirt, some: stone, grass, sand, sandstone, air, water

# Cycle consistency check

In [27]:
num_block_types = fqgan.in_channels # Or H.n_channels if fqgan.in_channels isn't an attribute

# --- Set model to evaluation mode ---
fqgan.eval()
device = 'cuda'

# --- Initialize accumulators ---
total_loss_pre_quant_style = 0.0
total_loss_pre_quant_struct = 0.0
total_loss_post_quant_conv_style = 0.0
total_loss_post_quant_conv_struct = 0.0
total_style_accuracy = 0.0
total_struct_accuracy = 0.0
num_batches = 0

print("Starting cycle consistency calculation...")

with torch.no_grad(): # Ensure no gradients are computed during this process
    for batch_idx, batch_data in enumerate(train_loader):
        # Assuming your train_loader yields data directly, or (data, labels)
        if isinstance(batch_data, (list, tuple)):
            input_chunks = batch_data[0].to(device)
        else:
            input_chunks = batch_data.to(device)

        # --- 1. First Pass: Encode original input ---
        h_style_orig_raw, h_struct_orig_raw = fqgan.encoder(input_chunks)

        h_style_orig_quant_conv = fqgan.quant_conv_style(h_style_orig_raw)
        h_struct_orig_quant_conv = fqgan.quant_conv_struct(h_struct_orig_raw)

        quant_style_orig, _, style_stats_orig = fqgan.quantize_style(h_style_orig_quant_conv)
        quant_struct_orig, _, struct_stats_orig = fqgan.quantize_struct(h_struct_orig_quant_conv)

        indices_style_orig = style_stats_orig[2].view(h_style_orig_quant_conv.shape[0], -1) # Flatten spatial
        indices_struct_orig = struct_stats_orig[2].view(h_struct_orig_quant_conv.shape[0], -1) # Flatten spatial


        # --- 2. Reconstruct ---
        quant_combined_orig = torch.cat([quant_struct_orig, quant_style_orig], dim=1)
        
        reconstructed_logits = None
        if fqgan.two_stage_decoder: # Check attribute on the model instance
            reconstructed_logits, _ = fqgan.decoder(quant_combined_orig)
        else:
            reconstructed_logits = fqgan.decoder(quant_combined_orig)

        # --- 3. Convert reconstructed logits to one-hot for re-encoding ---
        # The encoder expects one-hot encoded input, similar to `input_chunks`.
        # If your input_chunks are already float probabilities after one-hot,
        # then F.gumbel_softmax might be slightly closer. Otherwise, argmax + one_hot is standard.
        pred_indices = torch.argmax(reconstructed_logits, dim=1)
        reconstructed_one_hot = F.one_hot(pred_indices, num_classes=num_block_types)
        # Reshape from [B, H, W, D, C] to [B, C, H, W, D] for the Conv3D encoder
        reconstructed_one_hot = reconstructed_one_hot.permute(0, 4, 1, 2, 3).float()
        # No need to .detach() reconstructed_one_hot here as we are in torch.no_grad() context

        # --- 4. Second Pass: Encode reconstruction ---
        h_style_cycle_raw, h_struct_cycle_raw = fqgan.encoder(reconstructed_one_hot)

        h_style_cycle_quant_conv = fqgan.quant_conv_style(h_style_cycle_raw)
        h_struct_cycle_quant_conv = fqgan.quant_conv_struct(h_struct_cycle_raw)

        quant_style_cycle, _, style_stats_cycle = fqgan.quantize_style(h_style_cycle_quant_conv)
        quant_struct_cycle, _, struct_stats_cycle = fqgan.quantize_struct(h_struct_cycle_quant_conv)

        indices_style_cycle = style_stats_cycle[2].view(h_style_cycle_quant_conv.shape[0], -1) # Flatten spatial
        indices_struct_cycle = struct_stats_cycle[2].view(h_struct_cycle_quant_conv.shape[0], -1) # Flatten spatial

        # Calculate discrete accuracy
        style_accuracy = (indices_style_orig == indices_style_cycle).float().mean().item()
        struct_accuracy = (indices_struct_orig == indices_struct_cycle).float().mean().item()

        total_style_accuracy += style_accuracy
        total_struct_accuracy += struct_accuracy
        # --- 5. Calculate Cycle Consistency Losses ---
        
        # a) Pre-Quantization (comparing raw outputs of fqgan.encoder)
        loss_pre_quant_style = F.mse_loss(h_style_cycle_raw, h_style_orig_raw)
        loss_pre_quant_struct = F.mse_loss(h_struct_cycle_raw, h_struct_orig_raw)
        total_loss_pre_quant_style += loss_pre_quant_style.item()
        total_loss_pre_quant_struct += loss_pre_quant_struct.item()

        # b) Post-Quant_Conv (comparing after the 1x1 convs, before VQ)
        loss_post_quant_conv_style = F.mse_loss(h_style_cycle_quant_conv, h_style_orig_quant_conv)
        loss_post_quant_conv_struct = F.mse_loss(h_struct_cycle_quant_conv, h_struct_orig_quant_conv)
        total_loss_post_quant_conv_style += loss_post_quant_conv_style.item()
        total_loss_post_quant_conv_struct += loss_post_quant_conv_struct.item()
        
        num_batches += 1
        
        if (batch_idx + 1) % 50 == 0: # Print progress
            print(f"  Processed batch {batch_idx + 1}/{len(train_loader)}")
            current_avg_pre_style = (total_loss_pre_quant_style / num_batches)
            current_avg_pre_struct = (total_loss_pre_quant_struct / num_batches)
            current_avg_post_conv_style = (total_loss_post_quant_conv_style / num_batches)
            current_avg_post_conv_struct = (total_loss_post_quant_conv_struct / num_batches)
            current_avg_style_acc = (total_style_accuracy / num_batches)
            current_avg_struct_acc = (total_struct_accuracy / num_batches)
            print(f"    Avg Pre-Quant Loss (Style | Struct): {current_avg_pre_style:.4f} | {current_avg_pre_struct:.4f}")
            print(f"    Avg Post-Conv Loss (Style | Struct): {current_avg_post_conv_style:.4f} | {current_avg_post_conv_struct:.4f}")
            print(f"    Avg Discrete Accuracy (Style | Struct): {current_avg_style_acc:.4f} | {current_avg_struct_acc:.4f}")


# --- Calculate Averages ---
avg_loss_pre_quant_style = total_loss_pre_quant_style / num_batches if num_batches > 0 else 0
avg_loss_pre_quant_struct = total_loss_pre_quant_struct / num_batches if num_batches > 0 else 0
avg_loss_post_quant_conv_style = total_loss_post_quant_conv_style / num_batches if num_batches > 0 else 0
avg_loss_post_quant_conv_struct = total_loss_post_quant_conv_struct / num_batches if num_batches > 0 else 0
avg_total_style_accuracy = total_style_accuracy / num_batches if num_batches > 0 else 0
avg_total_struct_accuracy = total_struct_accuracy / num_batches if num_batches > 0 else 0

# --- Print Results ---
print("\n--- Average Cycle Consistency Losses ---")
print(f"Pre-Quantization Style Loss:   {avg_loss_pre_quant_style:.4f}")
print(f"Pre-Quantization Struct Loss:  {avg_loss_pre_quant_struct:.4f}")
print(f"Total Pre-Quantization Loss:   {(avg_loss_pre_quant_style + avg_loss_pre_quant_struct):.4f}")
print("-" * 30)
print(f"Post-Quant_Conv Style Loss:    {avg_loss_post_quant_conv_style:.4f}")
print(f"Post-Quant_Conv Struct Loss:   {avg_loss_post_quant_conv_struct:.4f}")
print(f"Total Post-Quant_Conv Loss:    {(avg_loss_post_quant_conv_style + avg_loss_post_quant_conv_struct):.4f}")
print("-" * 30)
print(f"Discrete Style Accuracy:    {avg_total_style_accuracy:.4f}")
print(f"Discrete Struct Accuracy:   {avg_total_struct_accuracy:.4f}")

# You could also consider comparing the quantized latents themselves:
# quant_style_cycle, _, _ = fqgan.quantize_style(h_style_cycle_quant_conv)
# quant_struct_cycle, _, _ = fqgan.quantize_struct(h_struct_cycle_quant_conv)
# loss_post_quant_style = F.mse_loss(quant_style_cycle, quant_style_orig)
# loss_post_quant_struct = F.mse_loss(quant_struct_cycle, quant_struct_orig)

Starting cycle consistency calculation...
  Processed batch 50/2502
    Avg Pre-Quant Loss (Style | Struct): 0.3912 | 0.3499
    Avg Post-Conv Loss (Style | Struct): 0.1241 | 0.1461
    Avg Discrete Accuracy (Style | Struct): 0.8165 | 0.7979
  Processed batch 100/2502
    Avg Pre-Quant Loss (Style | Struct): 0.3776 | 0.3317
    Avg Post-Conv Loss (Style | Struct): 0.1196 | 0.1383
    Avg Discrete Accuracy (Style | Struct): 0.8233 | 0.8056
  Processed batch 150/2502
    Avg Pre-Quant Loss (Style | Struct): 0.3858 | 0.3368
    Avg Post-Conv Loss (Style | Struct): 0.1219 | 0.1403
    Avg Discrete Accuracy (Style | Struct): 0.8210 | 0.8052
  Processed batch 200/2502
    Avg Pre-Quant Loss (Style | Struct): 0.3787 | 0.3264
    Avg Post-Conv Loss (Style | Struct): 0.1198 | 0.1361
    Avg Discrete Accuracy (Style | Struct): 0.8232 | 0.8079
  Processed batch 250/2502
    Avg Pre-Quant Loss (Style | Struct): 0.3770 | 0.3216
    Avg Post-Conv Loss (Style | Struct): 0.1194 | 0.1344
    Avg Discre

# Test the model

In [15]:
# Create an instnance of our visualizer
from visualization_utils import MinecraftVisualizerPyVista
visualizer = MinecraftVisualizerPyVista()


# Load a random sample from the training data

In [234]:
# Get a sample
# You can rerun this to get a different sample, since the train loader has shuffle=True
batch = next(iter(train_loader))
sample = batch[0].unsqueeze(0).cuda()  # Add batch dim and move to GPU

In [16]:
# If you find an interesting sample, like one with a cool feature or with a clear biome split, you can save it and later load it with the example code below
# torch.save(sample, 'steepriver2.pt')
sample = torch.load('shallowlake2.pt')

In [28]:
# Get style and structure code matrices
# Make sure to rerun this if you get a new sample from the loader
with torch.no_grad():
    style_indices, struct_indices = encode_and_quantize(fqgan, sample)
reconstructed, binary_reconstructed = decode_from_indices(style_indices, struct_indices, fqgan, two_stage=True)

In [19]:
# torch.save(style_indices, 'latents/flatocean2_styleindices.pt')
# torch.save(struct_indices, 'latents/flatocean2_structindices.pt')

In [18]:
# with torch.no_grad():
#     # Move indices to device only when needed
#     style_indices_gpu = style_indices.to('cuda')
#     struct_indices_gpu = struct_indices.to('cuda')
    
#     # Get quantized vectors
#     quant_style = fqgan.quantize_style.get_codebook_entry(
#         style_indices_gpu.view(-1),
#         shape=[1, fqgan.embed_dim, *style_indices.shape[1:]]
#     )
#     quant_struct = fqgan.quantize_struct.get_codebook_entry(
#         struct_indices_gpu.view(-1),
#         shape=[1, fqgan.embed_dim, *struct_indices.shape[1:]]
#     )
    
#     # Clear indices from GPU
#     del style_indices_gpu, struct_indices_gpu
    
#     # Combine and decode
#     quant = torch.cat([quant_struct, quant_style], dim=1)
#     # quant = quant_style + quant_struct
#     del quant_style, quant_struct
    
#     if True:
#         decoded, binary_decoded = fqgan.decoder(quant)
#     else:
#         decoded = fqgan.decoder(quant)
    
#     del quant
  

# Render the original dataset chunk

In [18]:
# batch = next(iter(train_loader))
# sample = batch[1].unsqueeze(0).cuda()

coords=[(0, 2, 5)]
# coords = coords + [(x, y, 0) for x in [3, 4] for y in range(6)]
# Don't forget: You need to convert from the one-hot encoded representaiton back into the original minecraft block IDs before rendering!
converted_orig = block_converter.convert_to_original_blocks(sample.squeeze())
plotter = visualizer.visualize_chunk(converted_orig, interactive=True, show_axis=False, highlight_latents=coords)
plotter.show()


    

Widget(value='<iframe src="http://localhost:57013/index.html?ui=P_0x1fca761e6a0_0&reconnect=auto" class="pyvis…

In [ ]:
# latent_coords = [(0, 2, 5)]
#   # Coordinates in the 6x6x6 latent space
# plotter = visualizer.visualize_latent_blocks(converted_orig, latent_coords, show_axis=False)
# plotter.show()

Widget(value='<iframe src="http://localhost:64892/index.html?ui=P_0x2b0a8cda5b0_1&reconnect=auto" class="pyvis…

In [29]:
# latent_coords = [(2, 2, 5)]  # Coordinates in the 6x6x6 latent space
# plotter = visualizer.visualize_isolated_latent_blocks(converted_orig, latent_coords, show_axis=False)
# plotter.reset_camera()
# plotter.show()
# img = plotter.screenshot(window_size=(500, 500), 
#                                transparent_background=True, 
#                                return_img=True)
# # Make the array C-contiguous and ensure correct format
# img_array = np.ascontiguousarray(img)

# # Save using imageio instead of plt.imsave
# import imageio
# imageio.imwrite('treesandcorner_225.png', img_array)
    

Widget(value='<iframe src="http://localhost:54255/index.html?ui=P_0x1b7424c2a30_7&reconnect=auto" class="pyvis…

# Render the reconstructed dataset chunk

In [19]:
# Reconstruct and convert from the style and structure matrices
binary_reconstructed = (binary_reconstructed>0.5).float()
binary_reconstructed = block_converter.convert_to_original_blocks(binary_reconstructed).squeeze()
reconstructed = block_converter.convert_to_original_blocks(reconstructed)

coords=[(0, 2, 5)]
# coords=None
fig = visualizer.visualize_chunk(reconstructed, highlight_latents=coords, show_axis=False, wireframe_highlight=True)
fig.show()

# fig.reset_camera()
# fig.show()
# img = fig.screenshot(window_size=(500, 500), 
#                                transparent_background=True, 
#                                return_img=True)
# # Make the array C-contiguous and ensure correct format
# img_array = np.ascontiguousarray(img)

# # Save using imageio instead of plt.imsave
# import imageio
# imageio.imwrite('treesandcorner_recon_2codebook_highlight.png', img_array)
    

Widget(value='<iframe src="http://localhost:57013/index.html?ui=P_0x1fc85b4d670_1&reconnect=auto" class="pyvis…

In [20]:
with torch.no_grad():
    reconstructed_raw = decode_from_indices(style_indices, struct_indices, fqgan, two_stage=True, return_raw=True)
    pred_indices_cycle = torch.argmax(reconstructed_raw, dim=1) 
    y_hard_cycle = F.one_hot(pred_indices_cycle, num_classes=fqgan.in_channels).permute(0, 4, 1, 2, 3).float()
    style_indices_cycle, struct_indices_cycle = encode_and_quantize(fqgan, y_hard_cycle)
    reconstructed_cycle, _ = decode_from_indices(style_indices_cycle, struct_indices_cycle, fqgan, two_stage=True)

reconstructed_cycle = block_converter.convert_to_original_blocks(reconstructed_cycle)

coords=[(0, 2, 5)]
# coords=None
fig = visualizer.visualize_chunk(reconstructed_cycle, highlight_latents=coords, show_axis=False, wireframe_highlight=True)
fig.show()

Widget(value='<iframe src="http://localhost:57013/index.html?ui=P_0x1fc85d4f640_2&reconnect=auto" class="pyvis…

In [21]:
plotter = visualizer.visualize_both_codes_with_blocks(
    style_indices_cycle,
    struct_indices_cycle,
    reconstructed_cycle
)
plotter.show(interactive=True)

In [35]:
# latent_coords = [(2, 2, 5)]  # Coordinates in the 6x6x6 latent space
# plotter = visualizer.visualize_isolated_latent_blocks(reconstructed, latent_coords, show_axis=False)
# plotter.reset_camera()
# plotter.show()
# img = plotter.screenshot(window_size=(500, 500), 
#                                transparent_background=True, 
#                                return_img=True)
# # Make the array C-contiguous and ensure correct format
# img_array = np.ascontiguousarray(img)

# # Save using imageio instead of plt.imsave
# import imageio
# imageio.imwrite('treesandcorner_recon_225.png', img_array)

Widget(value='<iframe src="http://localhost:54255/index.html?ui=P_0x1b754d91340_13&reconnect=auto" class="pyvi…

In [23]:
binary_reconstructed[binary_reconstructed != 5] = 217
fig = visualizer.visualize_chunk(binary_reconstructed, highlight_latents=coords)
fig.show()

Widget(value='<iframe src="http://localhost:61624/index.html?ui=P_0x23fee52a2e0_3&reconnect=auto" class="pyvis…

## VQ reconstruction (single codebook)

In [19]:
# Reconstruct and convert from the style and structure matrices
vq_reconstructed = vq_decode_from_indices(vq_latent_indices, vqgan)
vq_reconstructed = block_converter.convert_to_original_blocks(vq_reconstructed)
coords=None
fig = visualizer.visualize_chunk(vq_reconstructed, highlight_latents=coords, show_axis=False)
fig.show()

# fig.reset_camera()
# fig.show()
# img = fig.screenshot(window_size=(500, 500), 
#                                transparent_background=True, 
#                                return_img=True)
# # Make the array C-contiguous and ensure correct format
# img_array = np.ascontiguousarray(img)

# # Save using imageio instead of plt.imsave
# import imageio
# imageio.imwrite('treesandcorner_recon_singlecodebook.png', img_array)

Widget(value='<iframe src="http://localhost:55121/index.html?ui=P_0x27992c0df70_2&reconnect=auto" class="pyvis…

# Rotation experiment

In [22]:
def rotate_voxels_90(voxels, k=1):
    """
    Rotate voxels around Y axis by k*90 degrees
    Args:
        voxels: tensor of shape [B, C, H, W, D] or [C, H, W, D]
        k: number of 90 degree rotations (1 = 90°, 2 = 180°, 3 = 270°)
    Returns:
        Rotated voxels
    """
    # Handle both batched and unbatched inputs
    if len(voxels.shape) == 5:  # Batched [B, C, H, W, D]
        # Rotate around Y (height) axis by swapping width and depth dimensions
        return torch.rot90(voxels, k=k, dims=(2, 4))
    elif len(voxels.shape) == 4:  # Unbatched [C, H, W, D]
        return torch.rot90(voxels, k=k, dims=(1, 3))
    elif len(voxels.shape) == 3:  # Unbatched [H, W, D]
        return torch.rot90(voxels, k=k, dims=(0, 2))
    else:
        raise ValueError(f"Unexpected voxel shape: {voxels.shape}")

# Run experiment with original and rotated versions
def rotation_experiment(fqgan, terrain_sample):
    """
    Run rotation analysis on a terrain sample
    Returns encoded representations for original and rotated versions
    """
    results = {}
    rotations = [0, 1, 2, 3]
    
    for num_rot in rotations:
        # Rotate terrain if needed
        if num_rot == 0:
            rotated_terrain = terrain_sample
        else:
            rotated_terrain = rotate_voxels_90(terrain_sample, k=num_rot)
            
        rotated_terrain = rotated_terrain.to('cuda')
        
        # Get encodings from model
        with torch.no_grad():
            style_indices, struct_indices = encode_and_quantize(fqgan, rotated_terrain)
            reconstructed, binary_reconstructed = decode_from_indices(style_indices, struct_indices, fqgan, two_stage=True)
            reconstructed = block_converter.convert_to_original_blocks(reconstructed.squeeze())
        
        # unrotate everything
        if num_rot != 0:
            reconstructed = rotate_voxels_90(reconstructed, k=4-num_rot)
            style_indices = torch.rot90(style_indices, k=4-num_rot, dims=(1, 3))
            struct_indices = torch.rot90(struct_indices, k=4-num_rot, dims=(1, 3))
            
        # Store results
        results[num_rot] = {
            'structure': struct_indices,
            'style': style_indices,
            'recon': reconstructed
        }
    
    return results

def analyze_rotation_results(results):
    """
    Compare rotated versions against the original (unrotated) version
    Computes differences in reconstruction, style indices, and structure indices
    """
    # Get original versions as reference
    original_recon = results[0]['recon']
    original_style = results[0]['style']
    original_struct = results[0]['structure']
    
    print("Comparing rotated versions to original:")
    print("-" * 50)
    
    for rotation in [0, 1, 2, 3]:
        rotated_result = results[rotation]
        
        # Compare reconstructions (block differences)
        recon_diff = (original_recon != rotated_result['recon']).sum().item()
        total_blocks = original_recon.numel()
        
        # Compare style indices
        style_diff = (original_style != rotated_result['style']).sum().item()
        total_style = original_style.numel()
        
        # Compare structure indices
        struct_diff = (original_struct != rotated_result['structure']).sum().item()
        total_struct = original_struct.numel()
        
        print(f"\nRotation {rotation * 90}°:")
        print(f"Reconstruction differences: {recon_diff}/{total_blocks} blocks ({recon_diff/total_blocks*100:.2f}%)")
        print(f"Style code differences: {style_diff}/{total_style} indices ({style_diff/total_style*100:.2f}%)")
        print(f"Structure code differences: {struct_diff}/{total_struct} indices ({struct_diff/total_struct*100:.2f}%)")

def compute_rotation_change_heatmap(results):
    """
    Create heatmaps showing where style and structure codes change most frequently across rotations
    
    Args:
        results: Dictionary from rotation_experiment containing results for each rotation
        
    Returns:
        style_changes: 2D array showing frequency of style code changes at each position
        struct_changes: 2D array showing frequency of structure code changes at each position
    """
    # Get shape from original codes
    original_style = results[0]['style']
    original_struct = results[0]['structure']
    
    # Initialize count matrices
    # Remove batch and channel dimensions if present
    style_shape = original_style.squeeze().shape
    struct_shape = original_struct.squeeze().shape
    
    style_changes = torch.zeros(style_shape, device=original_style.device)
    struct_changes = torch.zeros(struct_shape, device=original_struct.device)
    
    # Compare each rotation to original
    for rotation in [1, 2, 3]:
        # Compare style codes
        style_diff = (original_style != results[rotation]['style'])
        style_changes += style_diff.squeeze().float()
        
        # Compare structure codes
        struct_diff = (original_struct != results[rotation]['structure'])
        struct_changes += struct_diff.squeeze().float()
    
    # Convert to CPU numpy arrays for visualization
    style_changes = style_changes.cpu().numpy()
    struct_changes = struct_changes.cpu().numpy()
    
    return style_changes, struct_changes

# Example usage:
# rotation_results = rotation_experiment(fqgan, terrain_sample)
# analyze_rotation_results(rotation_results)

# Example usage:
# Load a sample terrain (assuming you have this)
sample_terrain = sample  # Replace with actual terrain loading

# Run experiment
rotation_results = rotation_experiment(fqgan, sample_terrain)
analyze_rotation_results(rotation_results)
# Now you can analyze the results
# For example, to compare original vs 90-degree rotation:
# orig_structure = rotation_results[0]['structure']
# rot90_structure = rotation_results[90]['structure']

# You can implement comparison metrics here

Comparing rotated versions to original:
--------------------------------------------------

Rotation 0°:
Reconstruction differences: 0/13824 blocks (0.00%)
Style code differences: 0/216 indices (0.00%)
Structure code differences: 0/216 indices (0.00%)

Rotation 90°:
Reconstruction differences: 505/13824 blocks (3.65%)
Style code differences: 81/216 indices (37.50%)
Structure code differences: 78/216 indices (36.11%)

Rotation 180°:
Reconstruction differences: 461/13824 blocks (3.33%)
Style code differences: 91/216 indices (42.13%)
Structure code differences: 90/216 indices (41.67%)

Rotation 270°:
Reconstruction differences: 398/13824 blocks (2.88%)
Style code differences: 67/216 indices (31.02%)
Structure code differences: 61/216 indices (28.24%)


## Visualize where discrepencies are between rotations and original

In [48]:
style_heatmap, struct_heatmap = compute_rotation_change_heatmap(rotation_results)


In [49]:

plotter = visualizer.visualize_both_codes_with_blocks(
    style_heatmap,
    struct_heatmap,
    rotation_results[0]['recon']
)
plotter.show(interactive=True)

In [22]:
plotter = visualizer.visualize_chunk(rotation_results[0]['recon'], interactive=True, show_axis=False)
plotter.show()

Widget(value='<iframe src="http://localhost:57446/index.html?ui=P_0x2a4928838e0_2&reconnect=auto" class="pyvis…

In [23]:
plotter = visualizer.visualize_chunk(rotation_results[1]['recon'], interactive=True, show_axis=False)
plotter.show()

Widget(value='<iframe src="http://localhost:57446/index.html?ui=P_0x2a4929444f0_3&reconnect=auto" class="pyvis…

In [24]:
plotter = visualizer.visualize_chunk(rotation_results[2]['recon'], interactive=True, show_axis=False)
plotter.show()

Widget(value='<iframe src="http://localhost:57446/index.html?ui=P_0x2a492883670_4&reconnect=auto" class="pyvis…

In [25]:

plotter = visualizer.visualize_chunk(rotation_results[3]['recon'], interactive=True, show_axis=False)
plotter.show()

Widget(value='<iframe src="http://localhost:57446/index.html?ui=P_0x2a4928b7af0_5&reconnect=auto" class="pyvis…

## visualize latents of rotated

In [29]:
plotter = visualizer.visualize_both_codes_with_blocks(
    rotation_results[0]['style'],
    rotation_results[0]['structure'],
    rotation_results[0]['recon']
)
plotter.show(interactive=True)

In [23]:

plotter = visualizer.visualize_both_codes_with_blocks(
    rotation_results[1]['style'],
    rotation_results[1]['structure'],
    rotation_results[1]['recon']
)
plotter.show(interactive=True)

In [20]:

plotter = visualizer.visualize_both_codes_with_blocks(
    rotation_results[2]['style'],
    rotation_results[2]['structure'],
    rotation_results[2]['recon']
)
plotter.show(interactive=True)

In [29]:

plotter = visualizer.visualize_both_codes_with_blocks(
    rotation_results[3]['style'],
    rotation_results[3]['structure'],
    rotation_results[3]['recon']
)
plotter.show(interactive=True)

# Merging terrain experiment

In [31]:
# Function to merge two terrain samples, encode, and return merged sample and latents
def merge_terrain_samples(sample1, sample2, split_dim=0):
    # Create the merged sample as a new tensor instead of cloning sample1
    # Create the merged sample as a new tensor
    merged_sample = torch.zeros_like(sample1)
    
    # Adjust split_dim to account for batch and channel dimensions
    actual_dim = split_dim + 2  # +2 because of batch and channel dimensions
    
    # Get halfway point for splitting (using the spatial dimensions)
    half = sample1.shape[actual_dim] // 2
    
    # Split and merge based on the specified dimension
    if split_dim == 0:
        merged_sample[:, :, :half, :, :] = sample1[:, :, :half, :, :]
        merged_sample[:, :, half:, :, :] = sample2[:, :, half:, :, :]
    elif split_dim == 1:
        merged_sample[:, :, :, :half, :] = sample1[:, :, :, :half, :]
        merged_sample[:, :, :, half:, :] = sample2[:, :, :, half:, :]
    elif split_dim == 2:
        merged_sample[:, :, :, :, :half] = sample1[:, :, :, :, :half]
        merged_sample[:, :, :, :, half:] = sample2[:, :, :, :, half:]
    
    # Encode using existing model components
    struct_indices, style_indices = encode_and_quantize(fqgan, merged_sample)
    merged_sample = block_converter.convert_to_original_blocks(merged_sample.squeeze())
    
    merged_reconstructed, _ =  decode_from_indices(struct_indices, style_indices, fqgan, two_stage=True)
    merged_reconstructed = block_converter.convert_to_original_blocks(merged_reconstructed.squeeze())
    return merged_sample, merged_reconstructed, struct_indices, style_indices

# Function to merge latent representations and decode
def merge_latent_samples(struct_indices1, style_indices1, struct_indices2, style_indices2, split_dim=0):
    """
    Merges two sets of latent representations and decodes the result.
    
    Args:
        struct_indices1: Structure indices from first sample
        style_indices1: Style indices from first sample
        struct_indices2: Structure indices from second sample
        style_indices2: Style indices from second sample
        split_dim: Dimension to split along (0=x, 1=y, 2=z), default is 0
    
    Returns:
        reconstructed: The decoded merged latent representation
    """
    # Get halfway point for splitting latents (6x6x6)
    half = struct_indices1.shape[split_dim+1] // 2  # +1 because of batch dimension
    
    # Create the merged latents
    merged_struct = torch.zeros_like(struct_indices1)
    merged_style = torch.zeros_like(style_indices1)
    
    # Split and merge based on the specified dimension
    if split_dim == 0:
        merged_struct[:, :half, :, :] = struct_indices1[:, :half, :, :]
        merged_struct[:, half:, :, :] = struct_indices2[:, half:, :, :]
        merged_style[:, :half, :, :] = style_indices1[:, :half, :, :]
        merged_style[:, half:, :, :] = style_indices2[:, half:, :, :]
    elif split_dim == 1:
        merged_struct[:, :, :half, :] = struct_indices1[:, :, :half, :]
        merged_struct[:, :, half:, :] = struct_indices2[:, :, half:, :]
        merged_style[:, :, :half, :] = style_indices1[:, :, :half, :]
        merged_style[:, :, half:, :] = style_indices2[:, :, half:, :]
    elif split_dim == 2:
        merged_struct[:, :, :, :half] = struct_indices1[:, :, :, :half]
        merged_struct[:, :, :, half:] = struct_indices2[:, :, :, half:]
        merged_style[:, :, :, :half] = style_indices1[:, :, :, :half]
        merged_style[:, :, :, half:] = style_indices2[:, :, :, half:]
    
    # Decode the merged latents
    reconstructed, _ = decode_from_indices(merged_struct, merged_style, fqgan, two_stage=True)
    print(f'shape of reconstructed: {reconstructed.shape}')
    print(f'shape of struct: {merged_struct.shape}')
    print(f'shape of style: {merged_style.shape}')
    # Convert reconstructed data back to original block format
    reconstructed = block_converter.convert_to_original_blocks(reconstructed.squeeze())
    
    return reconstructed, merged_struct, merged_style

## Get two samples

In [32]:
sample1 = torch.load('latents/new_plateau.pt')
sample2 = torch.load('latents/new_stonemountainslope.pt')
sample1_conv = block_converter.convert_to_original_blocks(sample1.squeeze())
sample2_conv = block_converter.convert_to_original_blocks(sample2.squeeze())

## Merge the samples and pass through

In [33]:
fig = visualizer.visualize_chunk(sample1_conv, show_axis=False)
fig.show()

Widget(value='<iframe src="http://localhost:61320/index.html?ui=P_0x19213cadd60_7&reconnect=auto" class="pyvis…

In [34]:
fig = visualizer.visualize_chunk(sample2_conv, show_axis=False)
fig.show()

Widget(value='<iframe src="http://localhost:61320/index.html?ui=P_0x19213fc0430_8&reconnect=auto" class="pyvis…

In [35]:
merged_terrain, merged_reconstructed, merged_struct_indices, merged_style_indices = merge_terrain_samples(sample1, sample2, split_dim=1)
fig = visualizer.visualize_chunk(merged_terrain, show_axis=False)
fig.show()
fig = visualizer.visualize_chunk(merged_reconstructed, show_axis=False)
fig.show()

Widget(value='<iframe src="http://localhost:61320/index.html?ui=P_0x1921a687a60_9&reconnect=auto" class="pyvis…

Widget(value='<iframe src="http://localhost:61320/index.html?ui=P_0x1921a72c850_10&reconnect=auto" class="pyvi…

In [204]:
plotter = visualizer.visualize_both_codes_with_blocks(
    merged_style_indices,
    merged_struct_indices,
    merged_terrain
)
plotter.show(interactive=True)

## Merge the latents and pass through

In [36]:
s1_struct_indices, s1_style_indices = encode_and_quantize(fqgan, sample1)
s2_struct_indices, s2_style_indices = encode_and_quantize(fqgan, sample2)

In [37]:
mergedlatent_recon, mergedlatent_struct, mergedlatent_style = merge_latent_samples(s1_struct_indices, s1_style_indices, s2_struct_indices, s2_style_indices, split_dim=1)
print(mergedlatent_recon.shape)
fig = visualizer.visualize_chunk(mergedlatent_recon, show_axis=False)
fig.show()

shape of reconstructed: torch.Size([24, 24, 24])
shape of struct: torch.Size([1, 6, 6, 6])
shape of style: torch.Size([1, 6, 6, 6])
torch.Size([24, 24, 24])


Widget(value='<iframe src="http://localhost:61320/index.html?ui=P_0x1921d2af700_11&reconnect=auto" class="pyvi…

In [161]:
plotter = visualizer.visualize_both_codes_with_blocks(
    mergedlatent_style,
    mergedlatent_struct,
    mergedlatent_recon
)
plotter.show(interactive=True)

# Brush tool experiment

In [38]:
def shift_latent_slice(struct_indices, style_indices, slice_coords, shift_axis=1, shift_amount=1, shift_style=True):
    """
    Shifts a specified slice of the latents along specified axis.
    
    Args:
        struct_indices: Structure indices tensor [B, 6, 6, 6]
        style_indices: Style indices tensor [B, 6, 6, 6]
        slice_coords: Tuple of slices (x_slice, y_slice, z_slice) 
                     e.g., (slice(0,2), slice(0,3), slice(0,2))
        shift_axis: Axis to shift along (0=x, 1=y, 2=z), default is 1 (y-axis)
        shift_amount: Number of positions to shift (positive=up/right, negative=down/left)
        shift_style: Whether to apply the shift to style indices (default=True)
    """
    # Create copies to modify
    modified_struct = struct_indices.clone()
    modified_style = style_indices.clone()
    
    x_slice, y_slice, z_slice = slice_coords
    slices = [x_slice, y_slice, z_slice]
    
    # Get the original slice data
    slice_struct = modified_struct[:, x_slice, y_slice, z_slice].clone()
    slice_style = modified_style[:, x_slice, y_slice, z_slice].clone()
    
    # Get the values to duplicate (top or bottom depending on shift direction)
    fill_slices = list(slice_coords)
    if shift_amount > 0:
        # Moving up/right, so duplicate bottom values
        fill_slices[shift_axis] = slice(slices[shift_axis].start, slices[shift_axis].start + 1)
    else:
        # Moving down/left, so duplicate top values
        fill_slices[shift_axis] = slice(slices[shift_axis].stop - 1, slices[shift_axis].stop)
    
    fill_struct = modified_struct[:, fill_slices[0], fill_slices[1], fill_slices[2]].clone()
    fill_style = modified_style[:, fill_slices[0], fill_slices[1], fill_slices[2]].clone()
    
    # Calculate new coordinates after shift
    new_start = slices[shift_axis].start + shift_amount
    new_stop = slices[shift_axis].stop + shift_amount
    
    # Handle bounds for both positive and negative shifts
    if shift_amount > 0:
        valid_stop = min(new_stop, 6)
        valid_start = new_start
    else:
        valid_stop = new_stop
        valid_start = max(new_start, 0)
    
    valid_height = valid_stop - valid_start
    
    # Create new slices for the shifted position
    shifted_slices = list(slice_coords)
    shifted_slices[shift_axis] = slice(valid_start, valid_stop)
    
    # Calculate the offset in the source data based on shift direction
    if shift_amount > 0:
        source_start = 0
    else:
        source_start = -valid_height
    
    # Shift the slice
    modified_struct[:, shifted_slices[0], shifted_slices[1], shifted_slices[2]] = \
        slice_struct.narrow(shift_axis + 1, source_start, valid_height)
    modified_style[:, shifted_slices[0], shifted_slices[1], shifted_slices[2]] = \
        slice_style.narrow(shift_axis + 1, source_start, valid_height)
    
    # Fill in the gaps with duplicated values
    abs_shift = abs(shift_amount)
    for i in range(abs_shift):
        fill_slices = list(slice_coords)
        if shift_amount > 0:
            # Fill bottom gaps when shifting up
            pos = slices[shift_axis].start + i
        else:
            # Fill top gaps when shifting down
            pos = slices[shift_axis].stop - i - 1
        
        fill_slices[shift_axis] = slice(pos, pos + 1)
        modified_struct[:, fill_slices[0], fill_slices[1], fill_slices[2]] = fill_struct
        modified_style[:, fill_slices[0], fill_slices[1], fill_slices[2]] = fill_style
    
    # Decode using either modified or original style indices
    if shift_style:
        shifted_recon, _ = decode_from_indices(modified_struct, modified_style, fqgan, two_stage=True)
    else:
        shifted_recon, _ = decode_from_indices(modified_struct, style_indices, fqgan, two_stage=True)
    
    shifted_recon = block_converter.convert_to_original_blocks(shifted_recon.squeeze())
    
    return shifted_recon, modified_struct, modified_style
# Example usage:
"""
# Your slice [:2, :3, :2] would be passed in like this:
slice_coords = (slice(0,2), slice(0,3), slice(0,2))
modified_struct, modified_style = shift_latent_slice_up(struct_indices, style_indices, slice_coords)

# Reconstruct the modified version
reconstructed = decode_from_indices(modified_struct, modified_style, fqgan)
reconstructed = block_converter.convert_to_original_blocks(reconstructed)
"""

'\n# Your slice [:2, :3, :2] would be passed in like this:\nslice_coords = (slice(0,2), slice(0,3), slice(0,2))\nmodified_struct, modified_style = shift_latent_slice_up(struct_indices, style_indices, slice_coords)\n\n# Reconstruct the modified version\nreconstructed = decode_from_indices(modified_struct, modified_style, fqgan)\nreconstructed = block_converter.convert_to_original_blocks(reconstructed)\n'

In [39]:
shift_coords = [(x, y, z) for x in [2, 3] for y in  range(4) for z in  [2, 3]]
slice_coords = (slice(2, 4), slice(0,5), slice(2,4))


In [44]:
sample1 = torch.load('latents/new_bowlmountainlake.pt')
sample1_conv = block_converter.convert_to_original_blocks(sample1.squeeze())
s1_struct_indices, s1_style_indices = encode_and_quantize(fqgan, sample1)
fig = visualizer.visualize_chunk(sample1_conv, show_axis=False, highlight_latents=shift_coords, wireframe_highlight=True)
fig.show()

Widget(value='<iframe src="http://localhost:61320/index.html?ui=P_0x19235a0ac40_15&reconnect=auto" class="pyvi…

In [16]:
plotter = visualizer.visualize_both_codes_with_blocks(
    s1_style_indices,
    s1_struct_indices,
    sample1_conv
)
plotter.show(interactive=True)

In [41]:
shift_coords = [(x, y, z) for x in [3, 4, 5] for y in  [3, 4] for z in  [2, 3]]
slice_coords = (slice(3, 6), slice(3,5), slice(2,4))
# shift_coords = [(x, y, z) for x in [0, 1] for y in  range(4) for z in  [2, 3]]
# slice_coords = (slice(0, 2), slice(0,5), slice(2,4))


In [45]:
# Define your slice
# Shift the slice up
# Shift along y-axis (up) with style modification
shifted_recon, mod_struct, mod_style = shift_latent_slice(
    s1_struct_indices, 
    s1_style_indices, 
    slice_coords,
    shift_axis=0,  # y-axis
    shift_amount=2,
    shift_style=True
)
# shifted_recon, modified_struct, modified_style = shift_latent_slice_up(s1_struct_indices, s1_style_indices, slice_coords)
fig = visualizer.visualize_chunk(shifted_recon, show_axis=False, highlight_latents=shift_coords, wireframe_highlight=True)
fig.show()


Widget(value='<iframe src="http://localhost:61320/index.html?ui=P_0x19209db8d00_16&reconnect=auto" class="pyvi…

In [313]:
shifted_recon, modified_struct, modified_style = shift_latent_slice_up(s1_struct_indices, s1_style_indices, slice_coords)
fig = visualizer.visualize_chunk(shifted_recon, show_axis=False, highlight_latents=shift_coords, wireframe_highlight=True)
fig.show()

modified struct shape: torch.Size([1, 6, 6, 6])
modified style shape: torch.Size([1, 6, 6, 6])


Widget(value='<iframe src="http://localhost:54419/index.html?ui=P_0x1ba55b7b430_173&reconnect=auto" class="pyv…

# Plots either style or structure codes next to the chunk
NOTE: The slider can't take discrete values, if you slide it to layer 1 and nothing changes, keep moving the slider until you're past like 1.5, then it should change

In [41]:
# Visualize overlay
plotter = visualizer.visualize_latent_space_with_blocks(
    vq_latent_indices,  # Remove batch dimension
    vq_reconstructed,         # Remove batch dimension
    latent_type='style',
)
plotter.show()

Widget(value='<iframe src="http://localhost:55204/index.html?ui=P_0x1ff40642a00_17&reconnect=auto" class="pyvi…

In [38]:
# Visualize overlay
plotter = visualizer.visualize_latent_space_with_blocks(
    style_indices,  # Remove batch dimension
    reconstructed,         # Remove batch dimension
    latent_type='style',
)
plotter.show()

IndexError: index 4 is out of bounds for axis 2 with size 4

# Plot both style and structure next to the chunk
NOTE: This runs very slowly in notebook mode, so I'm rendering it a new window. Check to see if it opened.

In [23]:
plotter = visualizer.visualize_both_codes_with_blocks(
    style_indices,
    struct_indices,
    reconstructed
)
plotter.show(interactive=True)

# Make to change style indices, struct indices, or both
Pass "coords" into this to specify the range of spatial coordinates you want to make modifications to. This will simply loop from 0 to max_code_value + 1 and set all of the values at those coordinates equal to that value

NOTE: It currently only handles one max_code_value, so if your codebooks are different sizes, pass in the smaller value. If you try to go out of bounds, the CUDA kernel crashes and you must restart the kernel

In [24]:
def modify_latent_indices(indices, coords, code_value, max_code_value, mode='all'):
    """
    Helper function to modify latent indices either all to the same value or randomly.
    
    Args:
        indices: Original indices tensor [1,6,6,6]
        coords: List of (x,y,z) coordinates to modify
        code_value: Value to set for 'all' mode, or seed for random mode
        max_code_value: Maximum possible code value
        mode: Either 'all' or 'random'
    
    Returns:
        Modified indices tensor
    """
    modified = indices.clone()
    
    if mode == 'all':
        # Set all specified coordinates to the same value
        for coord in coords:
            modified[0, coord[0], coord[1], coord[2]] = code_value
    elif mode == 'random':
        # Set a different random value for each coordinate
        # Use the code_value as a seed for reproducibility
        torch.manual_seed(code_value)
        for coord in coords:
            modified[0, coord[0], coord[1], coord[2]] = torch.randint(0, max_code_value + 1, (1,)).item()
    
    return modified



def create_latent_modification_gif(fqgan, style_indices, struct_indices, coords, block_converter, 
                                 latent_type='style', max_code_value=31, duration=5.0, out_path='my_animation.gif', 
                                 show_axis=True, transparent_background=False, fps=3, mode='all', wireframe_highlight=True):
    """
    Creates a GIF showing how modifying style or structure codes at specific coordinates affects reconstruction.
    
    Args:
        fqgan: The trained FQGAN model
        style_indices: Original style indices tensor [1,6,6,6]
        struct_indices: Original structure indices tensor [1,6,6,6]
        coords: List of (x,y,z) coordinates to modify
        block_converter: BlockBiomeConverter instance for converting to block IDs
        latent_type: Either 'style' or 'struct' to specify which codes to modify
        max_code_value: Maximum value to try for codes
        duration: Duration of output GIF in seconds
        out_path: Path to save the output GIF
        mode: Either 'all' or 'random' - determines how codes are modified
    """
    chunks = []
    
    # First get base reconstruction
    base_reconstruction, _ = decode_from_indices(style_indices, struct_indices, fqgan, two_stage=True)
    base_reconstruction = block_converter.convert_to_original_blocks(base_reconstruction)
    chunks.append(base_reconstruction)
    
    # For each possible code value
    for code in range(max_code_value + 1):
        if latent_type == 'style':
            modified_style = modify_latent_indices(style_indices, coords, code, max_code_value, mode)
            modified_struct = struct_indices
            reconstruction, _ = decode_from_indices(modified_style, modified_struct, fqgan, two_stage=True)
            
        elif latent_type == 'both':
            modified_style = modify_latent_indices(style_indices, coords, code, max_code_value, mode)
            modified_struct = modify_latent_indices(struct_indices, coords, code, max_code_value, mode)
            reconstruction, _ = decode_from_indices(modified_style, modified_struct, fqgan, two_stage=True)
            
        else:  # struct
            modified_style = style_indices
            modified_struct = modify_latent_indices(struct_indices, coords, code, max_code_value, mode)
            reconstruction, _ = decode_from_indices(modified_style, modified_struct, fqgan, two_stage=True)
        
        # Convert to blocks and add to chunks
        reconstruction = block_converter.convert_to_original_blocks(reconstruction)
        chunks.append(reconstruction)
    
    # Create visualization
    visualizer = MinecraftVisualizerPyVista()
    visualizer.create_voxel_gif(chunks, duration=duration, output_path=out_path, 
                              highlight_latents=coords, show_axis=show_axis, 
                              transparent_background=transparent_background, fps=fps, wireframe_highlight=wireframe_highlight)
    
    return out_path

In [27]:

def create_latent_modification_gif_single_codebook(vqgan, indices, coords, block_converter, max_code_value=31, duration=5.0, out_path='my_animation.gif', show_axis=True, transparent_background=False, fps=3, wireframe_highlight=True, mode='all'):
    chunks = []
    
    # First get base reconstruction
    base_reconstruction = vq_decode_from_indices(indices, vqgan)
    base_reconstruction = block_converter.convert_to_original_blocks(base_reconstruction)
    chunks.append(base_reconstruction)
    
    # For each possible code value
    for code in range(max_code_value + 1):
        # modified_indices = indices.clone()
        modified_indices = modify_latent_indices(indices, coords, code, max_code_value, mode)
        # Modify the structure codes at specified coordinates
        # for coord in coords:
        #     modified_indices[0, coord[0], coord[1], coord[2]] = code
            
        reconstruction = vq_decode_from_indices(modified_indices, vqgan)
        
        # Convert to blocks and add to chunks
        reconstruction = block_converter.convert_to_original_blocks(reconstruction)
        chunks.append(reconstruction)
    
    # Create visualization
    visualizer = MinecraftVisualizerPyVista()
    visualizer.create_voxel_gif(chunks, duration=duration, output_path=out_path, highlight_latents=coords, show_axis=show_axis, transparent_background=transparent_background, fps=fps, wireframe_highlight=wireframe_highlight)
    
    return out_path

# Create style modification gif

In [29]:
#  Pick a name for this sample
sample_name = 'shallowlake_cyclecons_4_5k'

# Specify coordinate you want to modify
coord_list = []
coord_list.append([(x, y, z) for x in [1, 2] for y in  range(4) for z in  [1, 2]] ) # central vertical slice
coord_list.append([(x, y, z) for x in range(4) for y in [1, 2] for z in  [1, 2]] ) # central slice across x
coord_list.append([(x, y, z) for x in [1, 2] for y in [1, 2] for z in  range(4)] ) # central slice across x
# coord_list.append([(x, y, z) for x in [0, 1] for y in  range(4) for z in  [0, 1]] )
coord_list.append([(0, 2, 5)])
coord_list.append([(x, y, z) for x in range(6) for y in range(6) for z in range(6)] )
coord_list.append([(x, y, z) for x in [4, 5] for y in range(6) for z in [0, 1]] )
coord_list.append([(x, y, z) for x in [0, 1, 3] for y in [1, 2, 3, 4] for z in [0, 1, 2, 3]])
coord_list.append([(x, y, z) for x in [3, 4, 5] for y in range(6) for z in [4, 5]] )
coord_list.append([(x, y, z) for x in [0] for y in range(6) for z in [4, 5]] )
coord_list.append([(x, y, z) for x in [0, 1] for y in range(6) for z in [4, 5]] )


# coord_list.append([(x, y, z) for x in [2, 3] for y in  range(6) for z in  [2, 3]] ) # central vertical slice
# coord_list.append([(x, y, z) for x in range(6) for y in [2, 3] for z in  [2, 3]] ) # central slice across x
# coord_list.append([(x, y, z) for x in [2, 3] for y in [2, 3] for z in  range(6)] ) # central slice across x
# coord_list.append([(x, y, z) for x in [0, 1, 2] for y in  range(6) for z in  [0, 1]] )
# coord_list.append([(x, y, z) for x in range(6) for y in range(6) for z in range(6)] )
# coord_list.append([(x, y, z) for x in [4, 5] for y in range(6) for z in [0, 1]] )
# coord_list.append([(x, y, z) for x in [0, 1, 3] for y in [1, 2, 3, 4] for z in [0, 1, 2, 3]])
# coord_list.append([(x, y, z) for x in [3, 4, 5] for y in range(6) for z in [4, 5]] )
# coord_list.append([(x, y, z) for x in [0] for y in range(6) for z in [4, 5]] )
# coord_list.append([(x, y, z) for x in [0, 1] for y in range(6) for z in [4, 5]] )




max_style_code_value = fqgan_hparams.style_codebook_size - 1
max_struct_code_value = fqgan_hparams.struct_codebook_size - 1

for i, coords in enumerate(coord_list):
    # Run the gif creation
    gif_path = create_latent_modification_gif(
        fqgan,
        style_indices,
        struct_indices,
        coords,
        block_converter,
        latent_type='style',
        max_code_value=max_style_code_value,  # Assuming 32 possible codes (0-31)
        duration=100.0,  # 5 second animation,
        out_path=f'visualizations/{sample_name}_{i}_style_change.gif',
        transparent_background=True,
        show_axis=False,
        fps=1,
        # mode='random'
        mode='all'
    )

    gif_path = create_latent_modification_gif(
        fqgan,
        style_indices,
        struct_indices,
        coords,
        block_converter,
        latent_type='struct',
        max_code_value=max_struct_code_value,  # Assuming 32 possible codes (0-31)
        duration=10.0,  # 5 second animation,
        out_path=f'visualizations/{sample_name}_{i}_struct_change.gif',
        transparent_background=True,
        show_axis=False,
        fps=1,
        # mode='random'
        mode='all'
    )

# Create structure modification gif

In [ ]:
gif_path = create_latent_modification_gif(
    fqgan,
    style_indices,
    struct_indices,
    coords,
    block_converter,
    latent_type='struct',
    max_code_value=31,  # Assuming 32 possible codes (0-31)
    duration=10.0,  # 5 second animation,
    out_path=f'visualizations/{sample_name}_struct_change.gif',
)

torch.Size([1, 43, 24, 24, 24])
torch.Size([1, 1, 24, 24, 24])
torch.Size([1, 43, 24, 24, 24])
torch.Size([1, 1, 24, 24, 24])
torch.Size([1, 43, 24, 24, 24])
torch.Size([1, 1, 24, 24, 24])
torch.Size([1, 43, 24, 24, 24])
torch.Size([1, 1, 24, 24, 24])
torch.Size([1, 43, 24, 24, 24])
torch.Size([1, 1, 24, 24, 24])
torch.Size([1, 43, 24, 24, 24])
torch.Size([1, 1, 24, 24, 24])
torch.Size([1, 43, 24, 24, 24])
torch.Size([1, 1, 24, 24, 24])
torch.Size([1, 43, 24, 24, 24])
torch.Size([1, 1, 24, 24, 24])
torch.Size([1, 43, 24, 24, 24])
torch.Size([1, 1, 24, 24, 24])
torch.Size([1, 43, 24, 24, 24])
torch.Size([1, 1, 24, 24, 24])
torch.Size([1, 43, 24, 24, 24])
torch.Size([1, 1, 24, 24, 24])
torch.Size([1, 43, 24, 24, 24])
torch.Size([1, 1, 24, 24, 24])
torch.Size([1, 43, 24, 24, 24])
torch.Size([1, 1, 24, 24, 24])
torch.Size([1, 43, 24, 24, 24])
torch.Size([1, 1, 24, 24, 24])
torch.Size([1, 43, 24, 24, 24])
torch.Size([1, 1, 24, 24, 24])
torch.Size([1, 43, 24, 24, 24])
torch.Size([1, 1, 24, 2

Total frames: 33
Frame duration: 0.30303030303030304 seconds


# Create both modification gif

In [28]:
gif_path = create_latent_modification_gif(
    fqgan,
    style_indices,
    struct_indices,
    coords,
    block_converter,
    latent_type='both',
    max_code_value=19,  # Assuming 32 possible codes (0-31)
    duration=10.0,  # 5 second animation,
    out_path=f'visualizations/{sample_name}_both_change.gif',
)

torch.Size([1, 43, 24, 24, 24])
torch.Size([1, 1, 24, 24, 24])
torch.Size([1, 43, 24, 24, 24])
torch.Size([1, 1, 24, 24, 24])
torch.Size([1, 43, 24, 24, 24])
torch.Size([1, 1, 24, 24, 24])
torch.Size([1, 43, 24, 24, 24])
torch.Size([1, 1, 24, 24, 24])
torch.Size([1, 43, 24, 24, 24])
torch.Size([1, 1, 24, 24, 24])
torch.Size([1, 43, 24, 24, 24])
torch.Size([1, 1, 24, 24, 24])
torch.Size([1, 43, 24, 24, 24])
torch.Size([1, 1, 24, 24, 24])
torch.Size([1, 43, 24, 24, 24])
torch.Size([1, 1, 24, 24, 24])
torch.Size([1, 43, 24, 24, 24])
torch.Size([1, 1, 24, 24, 24])
torch.Size([1, 43, 24, 24, 24])
torch.Size([1, 1, 24, 24, 24])
torch.Size([1, 43, 24, 24, 24])
torch.Size([1, 1, 24, 24, 24])
torch.Size([1, 43, 24, 24, 24])
torch.Size([1, 1, 24, 24, 24])
torch.Size([1, 43, 24, 24, 24])
torch.Size([1, 1, 24, 24, 24])
torch.Size([1, 43, 24, 24, 24])
torch.Size([1, 1, 24, 24, 24])
torch.Size([1, 43, 24, 24, 24])
torch.Size([1, 1, 24, 24, 24])
torch.Size([1, 43, 24, 24, 24])
torch.Size([1, 1, 24, 2

Total frames: 21
Frame duration: 0.47619047619047616 seconds


# Create single codebook modification gif

In [33]:
# coords = [(x, y, z) for x in [0, 1] for y in range(6) for z in [4, 5]] 
sample_name = 'treesandcorner_asad_vqgan_rand_1_wireframe'
coords = [(x, y, z) for x in [0, 1] for y in range(6) for z in [4, 5]] 
# coords = [(x, y, z) for x in [3, 4, 5] for y in range(4) for z in [4, 5]] 
gif_path = create_latent_modification_gif_single_codebook(
    vqgan,
    vq_latent_indices,
    coords,
    block_converter,
    max_code_value=100,  # Assuming 32 possible codes (0-31)
    duration=20.0,  # 5 second animation,
    out_path=f'visualizations/{sample_name}_vqgan_change.gif',
    transparent_background=True,
    show_axis=False,
    fps=1,
    mode='random'
)